In [1]:
import pandas as pd
import math
import random
import numpy as np

In [2]:
breast_cancer = pd.read_csv("./breast-cancer_csv.csv")
breast_cancer.head(10)

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,40-49,premeno,15-19,0-2,yes,3,right,left_up,no,recurrence-events
1,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
2,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
3,40-49,premeno,35-39,0-2,yes,3,right,left_low,yes,no-recurrence-events
4,40-49,premeno,30-34,3-5,yes,2,left,right_up,no,recurrence-events
5,50-59,premeno,25-29,3-5,no,2,right,left_up,yes,no-recurrence-events
6,50-59,ge40,40-44,0-2,no,3,left,left_up,no,no-recurrence-events
7,40-49,premeno,10-14,0-2,no,2,left,left_up,no,no-recurrence-events
8,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
9,40-49,ge40,40-44,15-17,yes,2,right,left_up,yes,no-recurrence-events


In [149]:
breast_cancer = pd.read_csv("./breast-cancer_csv.csv")
breast_cancer = breast_cancer.fillna(method='ffill')
breast_cancer=breast_cancer.drop('Class',axis=1)
#attributes and values
attributes = breast_cancer.columns
# set(breast_cancer[attributes[0]])
attributes_and_values = {}
for x in attributes:
    attributes_and_values[x] = set(breast_cancer[x])
for x in attributes_and_values:
    print(x,attributes_and_values[x])
attributes=attributes.drop('irradiat')

age {'60-69', '50-59', '30-39', '70-79', '20-29', '40-49'}
menopause {'ge40', 'lt40', 'premeno'}
tumor-size {'15-19', '50-54', '35-39', '40-44', '25-29', '0-4', '20-24', '30-34', '45-49', '5-9', '10-14'}
inv-nodes {'0-2', '9-11', '15-17', '3-5', '24-26', '12-14', '6-8'}
node-caps {'yes', 'no'}
deg-malig {1, 2, 3}
breast {'right', 'left'}
breast-quad {'left_low', 'left_up', 'right_low', 'central', 'right_up'}
irradiat {'yes', 'no'}


In [150]:
def total_entropy(data: pd.DataFrame, attribute, attribute_values = attributes_and_values):
    size = len(data)
    total_entropy = 0
    for value in attribute_values[attribute]:
        value_prob = len(data[data[attribute] == value]) / size
        total_entropy -= value_prob*np.log2(value_prob)
    return total_entropy
def calc_entropy(filtered_data:pd.DataFrame,attribute,attribute_values = attributes_and_values):
    size = len(filtered_data)
    entropy = 0
    for value in attribute_values[attribute]:
        value_count = len(filtered_data[filtered_data[attribute] == value])
        entropy_of_value = 0
        if value_count != 0 :
            prob_value = value_count / size
            entropy_of_value -= prob_value*np.log2(prob_value)
        entropy+=entropy_of_value
    return entropy

def information_gain(data:pd.DataFrame,attribute,target = 'irradiat',attribute_values=attributes_and_values):
    size = len(data)
    attribute_info = 0
    for value in attribute_values[attribute]:
        filtered_data = data[data[attribute] == value]
        value_count = len(filtered_data)
        value_entropy = calc_entropy(filtered_data,target,attribute_values)
        value_prob = value_count/size
        attribute_info += value_prob*value_entropy
    return total_entropy(data,target,attribute_values) - attribute_info

In [151]:
def most_info_gain(data:pd.DataFrame,attributes = None,target = 'irradiat',attributes_and_values = attributes_and_values):

    info_gain = {}
    for attr in attributes:
        info_gain[attr] = information_gain(data,attr,target,attributes_and_values)
    return max(info_gain,key= lambda x : info_gain[x])

algorithm code


In [153]:
class Node:
    def __init__(self) -> None:
        self.attr_value = None#value of prev_attr to get to this node
        self.prev_attr = None#what was the previous split attribute
        self.attr = None#value to split with next
        self.childs = {}#children if any
    def __str__(self,level=0) -> str:
        if self.attr:
            t = ""
        else :
            t = "target value is "
        ret = "-"*(level**2)+repr(self.prev_attr)+" "+t +repr(self.attr_value)+"\n"
        temp = self.childs.get("most common target value")
        if temp != None:
            ret+="-"*(level**2)+"most common target  value is "+repr(temp)+"\n"
        else:
            for child in self.childs:
                ret += self.childs[child].__str__(level+1)
        return ret

In [154]:
def predict(node:Node,entry):
    if len(node.childs) == 0:#if there are no childs
        return node.attr_value
    res = node.childs.get("most common target value")
    if res != None:
        return res
    value = entry[node.attr]
    return predict(node.childs[value],entry)

In [155]:
class DTClassifier:
    def __init__(self,data:pd.DataFrame,target,attributes_and_values = attributes_and_values) -> None:
        self.data = data
        self.attributes = data.columns.drop(target)
        self.target = target
        self.target_values = data[target]
        self.attributes_and_values = attributes_and_values
        self.entropy = total_entropy(self.data,self.target,attributes_and_values)
        self.node = None
    def train(self,k = 10):
        self.node = self._id3_recv(self.data,self.attributes,self.node,k)
    def _id3_recv(self,data:pd.DataFrame,attributes : pd.Index, node,k):
        if not node:
            node = Node() #init the node
        if len(data[self.target].value_counts()) == 1: #if the data is pure (all entries have the same target value)
            node.attr_value = data[self.target].iloc[0]
            return node
        if len(attributes) == 0:#if all the features are exhausted and we still have impure data
            node.attr_value = data[self.target].value_counts().idxmax()#get the most common value
            return node
        # print(attributes)
        if len(data) <= k :#if we have less than k entries, prune the tree and return most common value
            node.attr_value = data[self.target].value_counts().idxmax()
            return node
        best_next_attribute = most_info_gain(data,attributes,self.target,self.attributes_and_values)
        node.attr = best_next_attribute
        # print(attributes,best_next_attribute)
        for value in attributes_and_values[best_next_attribute]:
            child = Node()
            child.attr_value = value
            child.prev_attr = best_next_attribute
            node.childs[value] = child
            child_data = data[data[best_next_attribute] == value]
            if child_data.empty:
                child.childs["most common target value"] = data[self.target].value_counts().idxmax()
            else :
                attributes_new = attributes.drop(best_next_attribute)
                child = self._id3_recv(child_data,attributes_new,child,k)
        return node
    def test(self,test_data:pd.DataFrame):
        size = len(test_data)
        hits = 0.0
        for _,entry in test_data.iterrows():
            hits += predict(self.node,entry) == entry[self.target]
        return hits/size
            

In [159]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(breast_cancer,test_size=0.3)
train = train.reset_index()
train.drop('index',axis=1,inplace=True)
test = test.reset_index()
test.drop('index',axis=1,inplace=True)
tree = DTClassifier(train,'irradiat',attributes_and_values)
tree.train()
print(tree.test(test))
print(tree.test(train))
# print(tree.node)


0.7674418604651163
0.85
